## Step 1: Import the Importer utility
This utility is used to dynamically load different classes available in BRAILS

In [ ]:
from brails.utils import Importer

## Step 2: Define Target Location and Output File Paths
In this step, we define the geographic area of interest and specify where the output files will be saved.

- `LOCATION_NAME`: This is the name of the location for which we want to retrieve and analyze building and elevation data. In this case, it's Fort Myers Beach, FL.
- `INVENTORY_OUTPUT`: This is the path to the file where the processed building inventory augmented with elevation information will be stored. The output format is GeoJSON, which is widely used for storing geospatial vector data.
- `ELEVATION_SURFACE_OUTPUT`: This defines the file path for storing the elevation surface data of the entire region. This file will include a gridded or interpolated representation of elevation across the study area, which may be useful for visualizations or further spatial analysis.

In [ ]:
LOCATION_NAME = 'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'FortMyersInventory_Elevation.geojson'
ELEVATION_SURFACE_OUTPUT = 'FortMyersElevationSurface.geojson'

## Step 3: Create an instance of the Importer to dynamically load classes from BRAILS++

In [ ]:
importer = Importer()

## Step 4: Create a Region Boundary Object from the Location Name
In this step, we define the geographic boundary for the analysis based on the provided location name.

- A dictionary called `region_data` is created to specify the input type (`"locationName"`) and the actual location (`LOCATION_NAME`), which was defined earlier.
- We dynamically load the appropriate class (`RegionBoundary`) using the `importer` utility.
- We then instantiate a region boundary object using the location data. This object will later be used to extract building and elevation data within the defined geographic extent.

In [ ]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

## Step 5: Retrieve Building Inventory from the National Structure Inventory (NSI)
In this step, we load and use the `NSI_Parser` class to extract building inventory data for the defined region.

- The `NSI_Parser` class is dynamically imported using the `importer` utility.
- An instance of the class is created to interface with the National Structure Inventory (NSI) database.
- The `get_raw_data()` method is called with the `region_boundary_object` as input, which returns the raw building inventory data for the specified location.

In [ ]:
nsi_scraper = importer.get_class('NSI_Parser')()
nsi_inventory = nsi_scraper.get_raw_data(region_boundary_object)

## Step 6: Retrieve Building Footprints from OpenStreetMap Using the `USA_FootprintScraper`
In this step, we use the `USA_FootprintScraper` class to extract building footprints from OpenStreetMap (OSM) for the specified region.

- The scraper is initialized with units set to feet (`'ft'`), which will be used for any length-related measurements.
- The `get_footprints()` method is called with the `region_boundary_object`, enabling the extraction of building footprint geometries (e.g., polygons) that fall within the region.

In [ ]:
fema_usa_scraper = importer.get_class('USA_FootprintScraper')({'length': 'ft'})
fema_usa_inventory = fema_usa_scraper.get_footprints(region_boundary_object)

## Step 7: Filter NSI Inventory by Overlapping Footprints and Enrich with Additional Features
In this step, we refine the National Structure Inventory (NSI) data by:

- Filtering the NSI records to include only buildings that spatially overlap with the footprints extracted from OpenStreetMap. This ensures we focus on relevant structures within the region.
- Enriching the filtered NSI data with additional attributes such as occupancy type, building area, and other extended features to provide more detailed information for subsequent analysis.

The `get_filtered_data_given_inventory()` method is used for this purpose, where:

- The building footprints (`fema_usa_inventory`) serve as the spatial filter.
- The units are specified as feet (`'ft'`).
- `get_extended_features=True` triggers retrieval of additional descriptive features.
- `add_features=[]` means no extra custom features are requested at this stage.

In [ ]:
nsi_inventory = nsi_scraper.get_filtered_data_given_inventory(
    fema_usa_inventory,
    'ft',  # Units
    get_extended_features=True,  # Whether to retrieve additional features
    add_features=[]  # No additional custom features specified
)

## Step 8: Load the USGSElevationScraper Class to Enrich Assets with Elevation Information
In this step, we prepare to enhance our asset data by integrating elevation details from the USGS Elevation Point Query Service. This enrichment enables more accurate spatial analyses and modeling by adding vertical context to each asset.

- We instantiate the `USGSElevationScraper` class using the dynamic `importer` utility.
- This scraper will later be used to query elevation data for each asset’s geographic coordinates.
- Elevation data can be customized by specifying different statistical modes (e.g., mean, median) when performing queries.
- Using this enriched elevation information helps in applications such as flood risk assessment, terrain analysis, and infrastructure planning.

The following code initializes the scraper instance:

In [ ]:
usgs_elevation_scraper = importer.get_class('USGSElevationScraper')()

## Step 9: Define the Statistical Modes for Summarizing Elevation Data
In this step, we specify the different statistical modes that will be used to summarize the elevation data retrieved for each asset. Each mode offers a unique perspective on the terrain’s elevation characteristics, helping to capture variability and central tendencies:

- `centroid`: Elevation at the geometric center of the asset footprint.
- `all`: Elevation values for all sampled points within the footprint.
- `average`: Mean elevation across all sampled points.
- `min`: Lowest elevation value found within the footprint.
- `max`: Highest elevation value found within the footprint.
- `median`: Median elevation, representing the middle value of the data set.
- `stddev`: Standard deviation, indicating how much elevation varies within the footprint.

In [ ]:
modes = ['centroid', 'all', 'average', 'min', 'max', 'median', 'stddev']

## Step 10: Retrieve Elevation Data for Each Building Asset Using the Selected Statistical Modes
In this step, we enrich the building asset inventory by querying elevation data for each asset based on the previously defined statistical modes.

The `get_asset_elevation_data()` method of the `USGSElevationScraper` class is called with:

- `asset_inventory`: The filtered NSI building data to be enriched.
- `modes`: The list of statistical modes specifying how elevation data should be summarized.

This process adds new elevation-related fields to each building record, providing detailed vertical context for each asset.

In [ ]:
asset_inventory = usgs_elevation_scraper.get_asset_elevation_data(
    asset_inventory=nsi_inventory,
    modes=modes
)

## Step 11: Export the Updated Asset Inventory with Elevation Data to a GeoJSON File
In this step, we save the enriched building asset inventory, now containing elevation attributes, to a GeoJSON file.

- The `write_to_geojson()` method is called on the `asset_inventory` object to write the data to the path specified by `INVENTORY_OUTPUT`.
- This output file contains both the original asset details and the newly added elevation information.

In [ ]:
_ = asset_inventory.write_to_geojson(INVENTORY_OUTPUT)

## Step 12: Fetch Elevation Data for the Entire Region by Sampling 2000 Random Points
In this step, we obtain elevation data for the broader geographic region by sampling a large number of random points within the region boundary.

The `get_region_elevation_data()` method of the `USGSElevationScraper` is called with:

- `region`: The polygon or boundary object defining the target area.
- `num_points`: The number of random points to sample for elevation data.

In [ ]:
region_elevation_data = usgs_elevation_scraper.get_region_elevation_data(
    region=region_boundary_object,
    num_points=2000
)

## Step 13: Export the Region-Wide Elevation Surface Data to a GeoJSON File
In this step, we save the sampled elevation surface data for the entire region into a GeoJSON file. Here, The `write_to_geojson()` method is used to write the `region_elevation_data` object to the specified output path (i.e., `ELEVATION_SURFACE_OUTPUT`).

In [ ]:
_ = region_elevation_data.write_to_geojson(ELEVATION_SURFACE_OUTPUT)